In [1]:
from utils import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, TensorDataset, Subset
from sklearn.model_selection import KFold, train_test_split
from tqdm import tqdm
import torch_geometric
import torch_geometric.nn.conv as conv
import pandas as pd
# Load the .mat file
'''
 data_path = f"E:\data/dataClean-ICA3-{num}-T1.mat"
 num = 25,...,35
'''

'\n data_path = f"E:\\data/dataClean-ICA3-{num}-T1.mat"\n num = 25,...,35\n'

In [32]:
data = loadmat('E:\data/dataClean-ICA3-25-T1.mat')
sample, blocks = get_data_subject(data, sub=0, selection_critirium=None)
sha = sample.shape
re_sample = np.reshape(sample, (sha[0],sha[1],sha[2]*sha[3]), order='F').T
re_sample = np.moveaxis(re_sample, 1, 2)
re_sample = torch.Tensor(re_sample)

# Model Testing

## Create Labels for sample

In [27]:
labels = [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2]
filtered_labels = []
for b in blocks:
    filtered_labels.append(labels[b])

Y_sample = np.repeat(filtered_labels, 108)
Y_hot = torch.Tensor(np.eye(3)[Y_sample])

## Iteration 1

In this implementation each node is independent, the time agregation weights are new for each nodes and timesteps.

In [323]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        self.time_convs.append(nn.Conv1d(in_channels=self.n_nodes, out_channels=self.n_nodes, kernel_size=T))
        self.dense = torch.nn.Linear(in_features= self.n_nodes, out_features=3)
        #for i in range(n_nodes):
        #    self.time_convs.append(nn.Conv1d(in_channels=1,
        #                                        out_channels=1,
        #                                        kernel_size=(1, T)))

    def forward(self, input):
        # input.shape [bs, n_nodes, 12000 (time_steps)]
        x = input
        
        #1D array of n_nodes channels
        #Time agregation convolution
        feat = self.time_convs[0](x)
        feat = feat.view(-1, self.n_nodes)
        feat = torch.relu(feat)

        #Dense layer for out 3 dim (classifier)
        out = self.dense(feat)
        out = torch.sigmoid(out) #For now probabilities
        return out

In [324]:
def Trial_train(net, s = 10, num_epochs = 10):
    
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    num_nodes = 52
    model = net(device, num_nodes,1200)
    
    # Define your loss function
    criterion = nn.MSELoss()
    
    # Define your optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Assuming you have your input and target data
    input_data = re_sample[::s]
    target_data = Y_hot[::s]
    
    # Training loop
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(input_data)
        
        # Compute the loss
        loss = criterion(output, target_data)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Print training statistics
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    print('Finished Training')
    return model 

model = Trial_train(net)

Epoch [1/10], Loss: 0.2628
Epoch [2/10], Loss: 0.0301
Epoch [3/10], Loss: 0.0056
Epoch [4/10], Loss: 0.0052
Epoch [5/10], Loss: 0.0051
Epoch [6/10], Loss: 0.0051
Epoch [7/10], Loss: 0.0051
Epoch [8/10], Loss: 0.0051
Epoch [9/10], Loss: 0.0049
Epoch [10/10], Loss: 0.0033
Finished Training


In [303]:
model

net(
  (time_convs): ModuleList(
    (0): Conv1d(52, 52, kernel_size=(1200,), stride=(1,))
  )
  (dense): Linear(in_features=52, out_features=3, bias=True)
)

In [301]:
model.time_convs[0].weight.shape

torch.Size([52, 52, 1200])

In [302]:
model.dense.weight.shape

torch.Size([3, 52])

## Iteration 2

In this implemntation nodes still do not interact however they share the same weights for the time agregation.

In [304]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=T))
        self.dense = torch.nn.Linear(in_features= self.n_nodes, out_features=3)
        #for i in range(n_nodes):
        #    self.time_convs.append(nn.Conv1d(in_channels=1,
        #                                        out_channels=1,
        #                                        kernel_size=(1, T)))

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        feat = torch.empty(bs, self.n_nodes)
        for i in range(self.n_nodes):
            feat[:,i] = self.time_convs[0](x[:,i].view(bs,1,1200)).view(-1)
        feat = torch.relu(feat)

        #Dense layer for out 3 dim (classifier)
        out = self.dense(feat)
        out = torch.sigmoid(out) #For now probabilities
        return out

In [305]:
model = Trial_train(net)

Epoch [1/10], Loss: 0.2516
Epoch [2/10], Loss: 0.2452
Epoch [3/10], Loss: 0.2393
Epoch [4/10], Loss: 0.2338
Epoch [5/10], Loss: 0.2286
Epoch [6/10], Loss: 0.2236
Epoch [7/10], Loss: 0.2190
Epoch [8/10], Loss: 0.2143
Epoch [9/10], Loss: 0.2098
Epoch [10/10], Loss: 0.2054
Finished Training


In [306]:
model.time_convs[0].weight.shape

torch.Size([1, 1, 1200])

In [307]:
model.dense.weight.shape

torch.Size([3, 52])

## Iteration 3

In [26]:
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=8, padding=1)(re_sample[:2,0].view(-1,1,1200))
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=8, stride=6, padding=1)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=6, stride=5, padding=1)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5)(x1)
x1.shape

torch.Size([2, 1, 150])
torch.Size([2, 1, 25])
torch.Size([2, 1, 5])


torch.Size([2, 1, 1])

In [309]:
x1.size(-1)

1

In [310]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=8, stride=6, padding=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=6, stride=5, padding=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=5))
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)
        #for i in range(n_nodes):
        #    self.time_convs.append(nn.Conv1d(in_channels=1,
        #                                        out_channels=1,
        #                                        kernel_size=(1, T)))

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))
        feat = t_layer_out[-1].view(-1,self.n_nodes)

        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out = torch.sigmoid(out) #For now probabilities
        return out

In [311]:
model = net(device, num_nodes,1200)
model(re_sample[:2])

tensor([[0.4746, 0.4824, 0.5301],
        [0.4650, 0.4878, 0.5244]], grad_fn=<SigmoidBackward0>)

In [318]:
model = Trial_train(net, num_epochs=300)

Epoch [1/300], Loss: 0.2465
Epoch [2/300], Loss: 0.2449
Epoch [3/300], Loss: 0.2434
Epoch [4/300], Loss: 0.2419
Epoch [5/300], Loss: 0.2405
Epoch [6/300], Loss: 0.2391
Epoch [7/300], Loss: 0.2378
Epoch [8/300], Loss: 0.2364
Epoch [9/300], Loss: 0.2351
Epoch [10/300], Loss: 0.2339
Epoch [11/300], Loss: 0.2327
Epoch [12/300], Loss: 0.2315
Epoch [13/300], Loss: 0.2303
Epoch [14/300], Loss: 0.2292
Epoch [15/300], Loss: 0.2281
Epoch [16/300], Loss: 0.2270
Epoch [17/300], Loss: 0.2260
Epoch [18/300], Loss: 0.2249
Epoch [19/300], Loss: 0.2239
Epoch [20/300], Loss: 0.2230
Epoch [21/300], Loss: 0.2221
Epoch [22/300], Loss: 0.2212
Epoch [23/300], Loss: 0.2203
Epoch [24/300], Loss: 0.2195
Epoch [25/300], Loss: 0.2187
Epoch [26/300], Loss: 0.2179
Epoch [27/300], Loss: 0.2172
Epoch [28/300], Loss: 0.2165
Epoch [29/300], Loss: 0.2158
Epoch [30/300], Loss: 0.2151
Epoch [31/300], Loss: 0.2145
Epoch [32/300], Loss: 0.2138
Epoch [33/300], Loss: 0.2132
Epoch [34/300], Loss: 0.2127
Epoch [35/300], Loss: 0

In [320]:
model

net(
  (time_convs): ModuleList(
    (0): Conv1d(1, 1, kernel_size=(10,), stride=(8,), padding=(1,))
    (1): Conv1d(1, 1, kernel_size=(8,), stride=(6,), padding=(1,))
    (2): Conv1d(1, 1, kernel_size=(6,), stride=(5,), padding=(1,))
    (3): Conv1d(1, 1, kernel_size=(5,), stride=(1,))
  )
  (class_layer): Linear(in_features=52, out_features=3, bias=True)
)

In [321]:
10+8+6+5+3*52

185

## Iteration 4
To do:
- Residuals and skip chanels
- Graph structures
- Pooling

I have decided to focus on:

1. pooling and
2. internal nfature representation

In [33]:
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=8, padding=1)(re_sample[:2,0].view(-1,1,1200))
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=5, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=3, padding=0)(x1)
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=3, stride=None, padding=1, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=4, padding=0)(x1)
x1.shape

torch.Size([2, 1, 150])
torch.Size([2, 1, 30])
torch.Size([2, 1, 10])
torch.Size([2, 1, 4])


torch.Size([2, 2, 1])

In [33]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T, internal_rep=None):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        self.irep = internal_rep
        
        if self.irep==None:
             self.irep=[1,1,1]
            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=self.irep[0],
                                         kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=5, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=self.irep[0], out_channels=self.irep[1],
                                         kernel_size=3, stride=3, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=1, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=self.irep[1], out_channels=self.irep[2],
                                         kernel_size=4, padding=0))
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes*self.irep[2], out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))
        
        feat = t_layer_out[-1].view(-1,self.n_nodes*self.irep[2])
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

In [34]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
rep = None
model = net(device, 42, 1200, rep)
model(re_sample[:2]) 

tensor([[0.3328, 0.3579, 0.3093],
        [0.3320, 0.3611, 0.3068]], grad_fn=<SoftmaxBackward0>)

### Training

Implementar filtro ente 250 y 500 Hz. (muestra kHz). Para ver si mejora. De 80 a 95 % accuracy debería tener.

For generatin the labels it seems each block belongs to a class, we will try to get all calses

In [5]:
def Generate_data(re_sample, blocks):
    labels = [0, 0, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2]
    filtered_labels = []
    for b in blocks:
        filtered_labels.append(labels[b])
    
    #re_sample and blocks already loaded
    Y_sample = np.repeat(filtered_labels, 108)
    Y_hot = torch.Tensor(np.eye(3)[Y_sample])
    return Y_sample, Y_hot 

In [6]:
Y_sample, Y_hot = Generate_data(re_sample, blocks)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(re_sample, Y_hot, test_size=0.2, random_state=42)

# Create training and test datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader for training and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [100]:
num_epochs = 20
batch_size = 32
learning_rate = 0.001

# Initialize the model, loss function, and optimizer
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
rep = None
model = net(device, 42, 1200, rep)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 300

# Training loop
for epoch in tqdm(range(num_epochs), desc=f'Training Epoch {epoch+1}/{num_epochs}'):
    model.train()
    train_loss = 0.0
    all_preds = []
    all_labels = []
    
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.squeeze())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Collect predictions and true labels
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(torch.argmax(targets, dim=1).squeeze().cpu().numpy())

    # Calculate training metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    print(f'Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')

Training Epoch 150/300:   0%|▏                                                         | 1/300 [00:07<38:36,  7.75s/it]

Epoch 1, Training Loss: 0.22141786822766968, Accuracy: 0.3561776061776062, Precision: 0.42281426311983505, Recall: 0.35661447391042095, F1-Score: 0.22811317730831535


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Training Epoch 150/300:   1%|▍                                                         | 2/300 [00:15<39:05,  7.87s/it]

Epoch 2, Training Loss: 0.21719272795951727, Accuracy: 0.40444015444015446, Precision: 0.30131362889983576, Recall: 0.40516795865633076, F1-Score: 0.3131297146172918


Training Epoch 150/300:   1%|▌                                                         | 3/300 [00:26<44:52,  9.07s/it]

...

Epoch 298, Training Loss: 0.18403226904796832, Accuracy: 0.5511583011583011, Precision: 0.566857426080727, Recall: 0.5512885739289728, F1-Score: 0.5500824473418863


Training Epoch 150/300: 100%|███████████████████████████████████████████████████████▊| 299/300 [36:01<00:07,  7.32s/it]

Epoch 299, Training Loss: 0.1854457539139372, Accuracy: 0.555019305019305, Precision: 0.573404543991066, Recall: 0.5551226070832956, F1-Score: 0.553984561727023


Training Epoch 150/300: 100%|████████████████████████████████████████████████████████| 300/300 [36:08<00:00,  7.23s/it]

Epoch 300, Training Loss: 0.1863565914558642, Accuracy: 0.553088803088803, Precision: 0.5678974892682589, Recall: 0.5532181809705026, F1-Score: 0.551960085269735


In [106]:
# Calculate test metrics
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred, average=None)
recall = recall_score(y_test, pred, average=None)
f1 = f1_score(y_test, pred, average=None)

print(f'Test Loss: {test_loss / len(test_loader)}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')

Test Loss: 0.22629646874136394, Accuracy: 0.3923076923076923, Precision: [0.48387097 0.37931034 0.34146341], Recall: [0.34090909 0.50574713 0.32941176], F1-Score: [0.4        0.43349754 0.33532934]


In [103]:
torch.save(model.state_dict(), 'model_state.pth')

# Initialize the model architecture
#model = SimpleModel(input_size=10, hidden_size=20, output_size=3)

# Load the state dictionary
#model.load_state_dict(torch.load('model_state.pth'))
#model.eval()

#### Training/Test Func

In [3]:
def train_model(model, num_epochs = 300, batch_size = 32, learning_rate = 0.001):

    # Initialize the model, loss function, and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        all_preds = []
        all_labels = []
        
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    
            # Collect predictions and true labels
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(targets, dim=1).squeeze().cpu().numpy())
    
        # Calculate training metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro')
        recall = recall_score(all_labels, all_preds, average='macro')
        f1 = f1_score(all_labels, all_preds, average='macro')
        
        print(f'Epoch {epoch+1}, Training Loss: {train_loss / len(train_loader)}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')
    return model

In [4]:
def Eval_Model(model):
    
    model.eval()
    pred = model(X_test)
    pred = torch.argmax(pred, dim=1)
    y_Test = torch.argmax(y_test, dim=1)

    # Calculate test metrics
    accuracy = accuracy_score(y_Test, pred)
    precision = precision_score(y_Test, pred, average=None)
    recall = recall_score(y_Test, pred, average=None)
    f1 = f1_score(y_Test, pred, average=None)
    
    print(f' Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')
    return accuracy, precision, recall, f1

### Differet training instances

In [ ]:
rep = [3,2,1]
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200, rep)

model = train_model(model, num_epochs = 300, batch_size = 32, learning_rate = 0.001)

Training Epoch 1/300:   0%|                                                                    | 0/300 [00:00<?, ?it/s]C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Training Epoch 1/300:   0%|▏                                                           | 1/300 [00:09<45:03,  9.04s/it]

Epoch 1, Training Loss: 0.22401587439305853, Accuracy: 0.33204633204633205, Precision: 0.16265406357149476, Recall: 0.33232259950716286, F1-Score: 0.1821961084472059


Training Epoch 1/300:   1%|▍                                                           | 2/300 [00:18<44:55,  9.05s/it]

Epoch 2, Training Loss: 0.2224415188485926, Accuracy: 0.33976833976833976, Precision: 0.36880643070714214, Recall: 0.34005216523610504, F1-Score: 0.19484185266420948


Training Epoch 1/300:   1%|▌                                                           | 3/300 [00:27<45:03,  9.10s/it]

...

Epoch 274, Training Loss: 0.17492029116009222, Accuracy: 0.5694980694980695, Precision: 0.596549744501457, Recall: 0.569223077289182, F1-Score: 0.5704604422414127


Training Epoch 1/300:  92%|█████████████████████████████████████████████████████▏    | 275/300 [49:10<05:45, 13.84s/it]

Epoch 275, Training Loss: 0.17347692788550348, Accuracy: 0.5733590733590733, Precision: 0.6076713101979442, Recall: 0.5730319780797223, F1-Score: 0.5741393192472569


Training Epoch 1/300:  92%|█████████████████████████████████████████████████████▎    | 276/300 [49:24<05:34, 13.92s/it]

Epoch 276, Training Loss: 0.17213596064936032, Accuracy: 0.5694980694980695, Precision: 0.5970084770774776, Recall: 0.5692119397263591, F1-Score: 0.5700661711302621


In [ ]:
Eval_Model(model)

In [9]:
rep = [2,2,1]
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200, rep)

model = train_model(model, num_epochs = 300, batch_size = 32, learning_rate = 0.001)

Training Epoch 1/300:   0%|▏                                                         | 1/300 [00:13<1:05:28, 13.14s/it]

Epoch 1, Training Loss: 0.2289096965934291, Accuracy: 0.2905405405405405, Precision: 0.16713685492281075, Recall: 0.29159925851027974, F1-Score: 0.18112861662470348


Training Epoch 1/300:   1%|▍                                                         | 2/300 [00:26<1:06:53, 13.47s/it]

Epoch 2, Training Loss: 0.2251558814084891, Accuracy: 0.30694980694980695, Precision: 0.1987710577188622, Recall: 0.3081199713013497, F1-Score: 0.16246555614196798


Training Epoch 1/300:   1%|▌                                                         | 3/300 [00:39<1:05:43, 13.28s/it]

...

Training Epoch 1/300: 100%|█████████████████████████████████████████████████████████▊| 299/300 [51:44<00:09,  9.96s/it]

Epoch 299, Training Loss: 0.17588473585518924, Accuracy: 0.5579150579150579, Precision: 0.5897338783553883, Recall: 0.5581971814843198, F1-Score: 0.5594484929908128


Training Epoch 1/300: 100%|██████████████████████████████████████████████████████████| 300/300 [51:54<00:00, 10.38s/it]

Epoch 300, Training Loss: 0.1761286768949393, Accuracy: 0.555984555984556, Precision: 0.5921993857085135, Recall: 0.5562535796418302, F1-Score: 0.5580001420837476


NameError: name 'test_loss' is not defined

In [11]:
Eval_Model(model)

 Accuracy: 0.4153846153846154, Precision: [0.57575758 0.36879433 0.33962264], Recall: [0.43181818 0.59770115 0.21176471], F1-Score: [0.49350649 0.45614035 0.26086957]


(0.4153846153846154,
 array([0.57575758, 0.36879433, 0.33962264]),
 array([0.43181818, 0.59770115, 0.21176471]),
 array([0.49350649, 0.45614035, 0.26086957]))

In [35]:
rep = None
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200, rep)

model = train_model(model, num_epochs = 80, batch_size = 32, learning_rate = 0.001)

Eval_Model(model)

  1%|█                                                                                  | 1/80 [00:04<06:11,  4.70s/it]

Epoch 1, Training Loss: 0.22202067664175323, Accuracy: 0.361003861003861, Precision: 0.36738685614050826, Recall: 0.36091636564489243, F1-Score: 0.3514010094487945


  2%|██                                                                                 | 2/80 [00:09<06:06,  4.70s/it]

Epoch 2, Training Loss: 0.22141523117368872, Accuracy: 0.41795366795366795, Precision: 0.39201918841931443, Recall: 0.41701304001396083, F1-Score: 0.357791120487069


  4%|███                                                                                | 3/80 [00:14<06:04,  4.73s/it]

...

Epoch 79, Training Loss: 0.11466481367295439, Accuracy: 0.7615830115830116, Precision: 0.7620504901072308, Recall: 0.7617153967416801, F1-Score: 0.7607600732600733


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [06:25<00:00,  4.82s/it]

Epoch 80, Training Loss: 0.11445354783173764, Accuracy: 0.7615830115830116, Precision: 0.7619655730006164, Recall: 0.7617042591788573, F1-Score: 0.7609852231198199


 Accuracy: 0.7346153846153847, Precision: [0.70408163 0.78409091 0.71621622], Recall: [0.78409091 0.79310345 0.62352941], F1-Score: [0.74193548 0.78857143 0.66666667]


(0.7346153846153847,
 array([0.70408163, 0.78409091, 0.71621622]),
 array([0.78409091, 0.79310345, 0.62352941]),
 array([0.74193548, 0.78857143, 0.66666667]))

### On ica_3

In [18]:
sha = data['ic_data3'].shape
ica3 = np.reshape(data['ic_data3'], (sha[-1]*sha[2],sha[0],sha[1]))
ica3 = torch.Tensor(ica3)

In [19]:
ica3.size()

torch.Size([1296, 42, 1200])

In [20]:
Y_sample, Y_hot = Generate_data(ica3, blocks)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(ica3, Y_hot, test_size=0.2, random_state=42)

# Create training and test datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoader for training and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [21]:
rep = None
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200, rep)

model = train_model(model, num_epochs = 80, batch_size = 32, learning_rate = 0.001)

Eval_Model(model)

  0%|                                                                                           | 0/80 [00:00<?, ?it/s]C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|█                                                                                  | 1/80 [00:04<06:04,  4.61s/it]

Epoch 1, Training Loss: 0.22226811719663214, Accuracy: 0.333011583011583, Precision: 0.111003861003861, Recall: 0.3333333333333333, F1-Score: 0.166545981173063


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|██                                                                                 | 2/80 [00:11<07:50,  6.03s/it]

Epoch 2, Training Loss: 0.22226674177429892, Accuracy: 0.333011583011583, Precision: 0.111003861003861, Recall: 0.3333333333333333, F1-Score: 0.166545981173063


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  4%|███                                                                                | 3/80 [00:18<08:10,  6.38s/it]

...

Epoch 19, Training Loss: 0.22222892533649097, Accuracy: 0.33494208494208494, Precision: 0.11164736164736165, Recall: 0.3333333333333333, F1-Score: 0.16726922149915643


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 25%|████████████████████▌                                                             | 20/80 [02:28<07:02,  7.05s/it]

Epoch 20, Training Loss: 0.22224597587729944, Accuracy: 0.33494208494208494, Precision: 0.11164736164736165, Recall: 0.3333333333333333, F1-Score: 0.16726922149915643


 25%|████████████████████▌                                                             | 20/80 [02:28<07:25,  7.43s/it]


KeyboardInterrupt: 

## Iteration 5

Forget internal rep.

Resulta que añadir capas del mismo tamaño puede ser provechoso en lugar de añadir canales de representacion interna redundante. Probar eso.

1. Mas capas.
2. En frecuencias.
3. En señal filtrada por frecuencias.

In [ ]:
GraphConvolution()()

In [8]:
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=8, padding=1)(re_sample[:2,0].view(-1,1,1200))
x1 = nn.MaxPool1d(kernel_size=5, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=3, padding=0)(x1)
x1 = nn.MaxPool1d(kernel_size=3, stride=None, padding=1, dilation=1, return_indices=False, ceil_mode=False)(x1)
#x1 = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=4, padding=0)(x1)
x1.shape

torch.Size([2, 1, 4])

In [ ]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        
            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=5, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=3, stride=3, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=1, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, padding=0))
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))
        
        feat = t_layer_out[-1].view(-1,self.n_nodes)
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

## Iteration 6 graph structure

In [18]:
re_sample[:2,...,:4].shape

torch.Size([2, 42, 4])

In [15]:
import torch_geometric.nn.conv as conv
#x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=10, stride=8, padding=1)(re_sample[:2,0].view(-1,1,1200))
x = re_sample[0,...,:4]
x.shape
A = torch.rand(42,42)
A = A.to_sparse()
x1 = conv.GCNConv(4, 4)(x, A)
print(x.shape)
#x1 = nn.MaxPool1d(kernel_size=5, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
#x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=3, padding=0)(x1)
#x1 = nn.MaxPool1d(kernel_size=3, stride=None, padding=1, dilation=1, return_indices=False, ceil_mode=False)(x1)
#x1 = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=4, padding=0)(x1)

torch.Size([42, 4])


In [12]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        A = torch.rand(self.n_nodes,self.n_nodes)
        self.A1 = A.to_sparse()
        self.A2 = A.to_sparse()
            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=5, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=3, stride=3, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=1, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, padding=0))
        #Graph Conv.
        self.gcn1 = conv.GCNConv(30,30)
        self.gcn2 = conv.GCNConv(4,4)
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))

            if i==2:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn1(batches[b],self.A1)
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
                
            if i==4:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn2(batches[b],self.A2)
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
            
        feat = t_layer_out[-1].view(-1,self.n_nodes)
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

In [13]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200)
model(re_sample[:2]) 

tensor([[0.3858, 0.2368, 0.3774],
        [0.4070, 0.2367, 0.3563]], grad_fn=<SoftmaxBackward0>)

In [14]:
model = train_model(model, num_epochs = 300, batch_size = 32, learning_rate = 0.001)

  0%|▎                                                                                 | 1/300 [00:07<38:48,  7.79s/it]

Epoch 1, Training Loss: 0.22429469047170697, Accuracy: 0.3166023166023166, Precision: 0.3096237448543839, Recall: 0.31710057024969185, F1-Score: 0.24327678205988068


  1%|▌                                                                                 | 2/300 [00:15<38:47,  7.81s/it]

Epoch 2, Training Loss: 0.2223616143067678, Accuracy: 0.32432432432432434, Precision: 0.3166101437182365, Recall: 0.32413494484801925, F1-Score: 0.3083550153582871


  1%|▊                                                                                 | 3/300 [00:23<38:26,  7.77s/it]

...

Epoch 298, Training Loss: 0.08593848093666813, Accuracy: 0.8233590733590733, Precision: 0.8237528951459043, Recall: 0.823359338247459, F1-Score: 0.8232568132070619


100%|███████████████████████████████████████████████████████████████████████████████▋| 299/300 [41:21<00:08,  8.65s/it]

Epoch 299, Training Loss: 0.08651519126512787, Accuracy: 0.8185328185328186, Precision: 0.8192236142140077, Recall: 0.8185561671502116, F1-Score: 0.818028014144519


100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [41:30<00:00,  8.30s/it]

Epoch 300, Training Loss: 0.08574415901393602, Accuracy: 0.8223938223938224, Precision: 0.8222816187577034, Recall: 0.8224014835622199, F1-Score: 0.8220908571481499


In [25]:
model(X_test[-5:])

tensor([[9.9982e-01, 1.7224e-04, 3.3090e-06],
        [1.8925e-02, 1.7206e-03, 9.7935e-01],
        [8.4620e-05, 9.7188e-01, 2.8031e-02],
        [9.8204e-01, 1.7948e-02, 1.5297e-05],
        [4.4494e-03, 4.8090e-04, 9.9507e-01]], grad_fn=<SoftmaxBackward0>)

In [24]:
y_test[-5:]

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [15]:
Eval_Model(model)

 Accuracy: 0.7384615384615385, Precision: [0.82352941 0.75238095 0.65517241], Recall: [0.63636364 0.90804598 0.67058824], F1-Score: [0.71794872 0.82291667 0.6627907 ]


(0.7384615384615385,
 array([0.82352941, 0.75238095, 0.65517241]),
 array([0.63636364, 0.90804598, 0.67058824]),
 array([0.71794872, 0.82291667, 0.6627907 ]))

## Iteration 7, more layers smaller kernells

In [35]:
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=2, stride=2, padding=0)(re_sample[:2,0].view(-1,1,1200))
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=3, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1)(x1)
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1)(x1)
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=4, stride=2, padding=1)(x1)
print(x1.shape)
x1 = nn.MaxPool1d(kernel_size=2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)(x1)
print(x1.shape)
x1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)(x1)
x1.shape

torch.Size([2, 1, 600])
torch.Size([2, 1, 200])
torch.Size([2, 1, 100])
torch.Size([2, 1, 50])
torch.Size([2, 1, 25])
torch.Size([2, 1, 12])
torch.Size([2, 1, 6])
torch.Size([2, 1, 3])


torch.Size([2, 1, 1])

In [26]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        A = torch.rand(self.n_nodes,self.n_nodes)
        self.A1 = A.to_sparse()
        self.A2 = A.to_sparse()
            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=2, stride=2, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, stride=2, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=2, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, stride=2, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=2, stride=None,
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1, 
                                         kernel_size=4, stride=2, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=2, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=3))
        #Graph Conv.
        self.gcn1 = conv.GCNConv(50,50)
        self.gcn2 = conv.GCNConv(12,12)
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))
                
            if i%2==1:
                t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))
            else:
                t_layer_out.append(torch.stack(nodes, axis=1))

            if i==4:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn1(batches[b],self.A1)
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
                
            if i==6:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn2(batches[b],self.A2)
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
            
        feat = t_layer_out[-1].view(-1,self.n_nodes)
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

In [27]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200)
model(re_sample[:2]) 

tensor([[0.3544, 0.3329, 0.3127],
        [0.3544, 0.3329, 0.3127]], grad_fn=<SoftmaxBackward0>)

In [28]:
model = train_model(model, num_epochs = 80, batch_size = 32, learning_rate = 0.001)

  1%|█                                                                                  | 1/80 [00:20<26:31, 20.14s/it]

Epoch 1, Training Loss: 0.22273525008649536, Accuracy: 0.3185328185328185, Precision: 0.3158132428879383, Recall: 0.31788198036227516, F1-Score: 0.28077796902434815


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  2%|██                                                                                 | 2/80 [00:40<26:38, 20.50s/it]

Epoch 2, Training Loss: 0.22268752844044656, Accuracy: 0.333011583011583, Precision: 0.111003861003861, Recall: 0.3333333333333333, F1-Score: 0.166545981173063


  4%|███                                                                                | 3/80 [01:00<26:00, 20.26s/it]

...

Epoch 78, Training Loss: 0.16880674795670944, Accuracy: 0.6216216216216216, Precision: 0.6235155542132286, Recall: 0.6214946139846994, F1-Score: 0.6210423502600894


 99%|████████████████████████████████████████████████████████████████████████████████▉ | 79/80 [26:46<00:20, 20.64s/it]

Epoch 79, Training Loss: 0.16272606768391348, Accuracy: 0.637065637065637, Precision: 0.640130568066652, Recall: 0.6369566026807207, F1-Score: 0.6370584685603942


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [27:07<00:00, 20.34s/it]

Epoch 80, Training Loss: 0.15973724763501773, Accuracy: 0.6341698841698842, Precision: 0.6354928034075155, Recall: 0.6341360634607969, F1-Score: 0.6341326109149444


In [29]:
Eval_Model(model)

 Accuracy: 0.6461538461538462, Precision: [0.7962963  0.68       0.53773585], Recall: [0.48863636 0.7816092  0.67058824], F1-Score: [0.6056338  0.72727273 0.59685864]


(0.6461538461538462,
 array([0.7962963 , 0.68      , 0.53773585]),
 array([0.48863636, 0.7816092 , 0.67058824]),
 array([0.6056338 , 0.72727273, 0.59685864]))

## Iteration 8, make it simpler (just 1 graph conv)

In [2]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
        A = torch.rand(self.n_nodes,self.n_nodes)
        self.A = A.to_sparse()

            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=5, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=3, stride=3, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=1, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, padding=0))
        #Graph Conv.
        self.gcn1 = conv.GCNConv(4,4)
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))

            if i==4:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn1(batches[b],self.A)
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
            
        feat = t_layer_out[-1].view(-1,self.n_nodes)
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

In [31]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200)
model = train_model(model, num_epochs = 80, batch_size = 32, learning_rate = 0.001)

  0%|                                                                                           | 0/80 [00:00<?, ?it/s]C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  1%|█                                                                                  | 1/80 [00:06<08:59,  6.83s/it]

Epoch 1, Training Loss: 0.2246884690089659, Accuracy: 0.3359073359073359, Precision: 0.27852998065764023, Recall: 0.33430232558139533, F1-Score: 0.16943824739549543


  2%|██                                                                                 | 2/80 [00:13<08:34,  6.59s/it]

Epoch 2, Training Loss: 0.2223229737895908, Accuracy: 0.3137065637065637, Precision: 0.28267694063926946, Recall: 0.31232266102277145, F1-Score: 0.1824207309613355


  4%|███                                                                                | 3/80 [00:19<08:26,  6.58s/it]

...

Epoch 78, Training Loss: 0.1118452325463295, Accuracy: 0.7760617760617761, Precision: 0.7765216416236824, Recall: 0.7762085874494561, F1-Score: 0.7758306837314445


 99%|████████████████████████████████████████████████████████████████████████████████▉ | 79/80 [08:32<00:06,  6.44s/it]

Epoch 79, Training Loss: 0.11241704022342508, Accuracy: 0.7644787644787645, Precision: 0.7663004655715097, Recall: 0.7645723030181176, F1-Score: 0.764598367262204


100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [08:38<00:00,  6.48s/it]

Epoch 80, Training Loss: 0.10947213333212968, Accuracy: 0.7799227799227799, Precision: 0.7803665084915085, Recall: 0.7800734188788813, F1-Score: 0.7795872280126979


In [32]:
Eval_Model(model)

 Accuracy: 0.7461538461538462, Precision: [0.8125     0.71568627 0.71794872], Recall: [0.73863636 0.83908046 0.65882353], F1-Score: [0.77380952 0.77248677 0.68711656]


(0.7461538461538462,
 array([0.8125    , 0.71568627, 0.71794872]),
 array([0.73863636, 0.83908046, 0.65882353]),
 array([0.77380952, 0.77248677, 0.68711656]))

In [8]:
print(model.A)

tensor(indices=tensor([[ 0,  0,  0,  ..., 41, 41, 41],
                       [ 0,  1,  2,  ..., 39, 40, 41]]),
       values=tensor([0.6959, 0.8297, 0.3468,  ..., 0.1942, 0.2221, 0.5544]),
       size=(42, 42), nnz=1764, layout=torch.sparse_coo)


By default A is not adaptative

## Iteration 9 make it adaptble

In [26]:
class net(nn.Module):
    def __init__(self, device, num_nodes, T):
        super(net, self).__init__()
        self.time_convs = nn.ModuleList()
        self.T = T
        self.n_nodes = num_nodes
            
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=10, stride=8, padding=1))
        self.time_convs.append(nn.MaxPool1d(kernel_size=5, stride=None, 
                                            padding=0, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=3, stride=3, padding=0))
        self.time_convs.append(nn.MaxPool1d(kernel_size=3, stride=None, 
                                            padding=1, dilation=1))
        self.time_convs.append(nn.Conv1d(in_channels=1, out_channels=1,
                                         kernel_size=4, padding=0))
        #Graph Conv.
        self.gcn1 = AdaptiveGCNLayer(4, 4)
        
        self.class_layer = torch.nn.Linear(in_features= self.n_nodes, out_features=3)            

    def forward(self, input):
        # input.shape [bs, n_nodes, 1200 (time_steps)]
        x = input
        bs = input.size(0)
        
        #1D array of n_nodes channels
        #Time agregation convolution
        t_layer_out = [x]
        for i in range(len(self.time_convs)):
            nodes=[]
            for j in range(self.n_nodes):
                # Internally we are reshaping to the number of canels (automatically infered with -1)
                nodes.append(self.time_convs[i](t_layer_out[i][:,j].view(bs,-1,t_layer_out[i].size(-1))))   
            t_layer_out.append(torch.relu(torch.stack(nodes, axis=1)))

            if i==4:
                batches=[]
                for b in range(bs):
                    batches.append(t_layer_out[i].view(bs,self.n_nodes,-1)[b])
                    batches[b] = self.gcn1(batches[b])
                t_layer_out[i] = torch.stack(batches, axis=1).view(bs,self.n_nodes,1,-1)
                batches=[]
            
        feat = t_layer_out[-1].view(-1,self.n_nodes)
            
        #Dense layer for out 3 dim (classifier)
        out = self.class_layer(feat)
        out =  nn.Softmax(dim=1)(out) #  torch.sigmoid(out) #For now probabilities
        return out

In [27]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = net(device, 42, 1200)
model = train_model(model, num_epochs = 10, batch_size = 32, learning_rate = 0.001)

tensor(indices=tensor([[ 0,  0,  0,  ..., 41, 41, 41],
                       [ 0,  1,  2,  ..., 39, 40, 41]]),
       values=tensor([0.3144, 0.3149, 0.4175,  ..., 0.6345, 0.3149, 0.7913]),
       size=(42, 42), nnz=1764, layout=torch.sparse_coo)


 10%|████████▎                                                                          | 1/10 [00:05<00:50,  5.57s/it]

Epoch 1, Training Loss: 0.2225224393786806, Accuracy: 0.2953667953667954, Precision: 0.19709769078693296, Recall: 0.2950204409892098, F1-Score: 0.2348680156899335


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 20%|████████████████▌                                                                  | 2/10 [00:10<00:43,  5.44s/it]

Epoch 2, Training Loss: 0.2222116259914456, Accuracy: 0.33494208494208494, Precision: 0.23603322949117342, Recall: 0.3356622657105199, F1-Score: 0.2417309719586759


...

Epoch 8, Training Loss: 0.1934341968912067, Accuracy: 0.5714285714285714, Precision: 0.6015525066147996, Recall: 0.5708277038457292, F1-Score: 0.5538716612053073


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:48<00:05,  5.37s/it]

Epoch 9, Training Loss: 0.18664350608984628, Accuracy: 0.61003861003861, Precision: 0.632474496596482, Recall: 0.6096208550216324, F1-Score: 0.6043574246146767


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:53<00:00,  5.39s/it]

Epoch 10, Training Loss: 0.18199909862243768, Accuracy: 0.612934362934363, Precision: 0.642533357428993, Recall: 0.612496984925762, F1-Score: 0.6053555568798489


In [25]:
class AdaptiveGCNLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(AdaptiveGCNLayer, self).__init__()
        self.gcn =  conv.GCNConv(in_channels, out_channels)
        self.adj_weight = nn.Parameter(torch.FloatTensor(in_channels, in_channels))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.adj_weight)

    def forward(self, x):
        # Compute the adaptive adjacency matrix
        adj = torch.matmul(x, self.adj_weight)
        adj = torch.matmul(adj, x.transpose(1, 0))

        # Add self-loops to the adjacency matrix
        adj = adj + torch.eye(adj.size(0), device=adj.device)

        # Normalize the adjacency matrix
        row_sum = adj.sum(1)
        d_inv_sqrt = row_sum.pow(-0.5)
        d_mat_inv_sqrt = torch.diag(d_inv_sqrt)
        adj = torch.matmul(d_mat_inv_sqrt, adj)
        adj = torch.matmul(adj, d_mat_inv_sqrt)

        # Use the normalized adjacency matrix in the graph convolution
        edge_index, _ = torch_geometric.utils.dense_to_sparse(adj)
        return self.gcn(x, edge_index)

# Training Models

Some subjects will not work.

Me queda añadir guardado de metricas de entreno 

In [2]:
from model import *
from data import *

In [3]:
def train_model(model, DataClass, num_epochs = 300, batch_size = 32,
                learning_rate = 0.001, save=False, path=None, path_metrics=None, verbose=True):
    # Initialize the model, loss function, and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    ps = []
    recs = []
    accs = []
    f1s = []
    Loss = []
    for epoch in tqdm(range(num_epochs), disable=not(verbose)):
        model.train()
        train_loss = 0.0
        all_preds = []
        all_labels = []
        
        for inputs, targets in DataClass.train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets.squeeze())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
    
            # Collect predictions and true labels
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(torch.argmax(targets, dim=1).squeeze().cpu().numpy())
    
        # Calculate training metrics
        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro')
        recall = recall_score(all_labels, all_preds, average='macro')
        f1 = f1_score(all_labels, all_preds, average='macro')

        if verbose:
            print(f'Epoch {epoch+1}, Training Loss: {train_loss / len(DataClass.train_loader)}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')
        ps.append(precision)
        recs.append(recall)
        accs.append(accuracy)
        f1s.append(f1)
        Loss.append(train_loss / len(DataClass.train_loader))
        
    if save:
        torch.save(model.state_dict(), path)
        dic = {'Loss':Loss,'Accuracy':accs,'Recall':recs,'Precision':ps,'F1':f1s}
        df = pd.DataFrame.from_dict(dic)
        df.to_csv(path_metrics)
        return
        
    return model

In [10]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
DataClass = subjecData()
DataClass.build()

In [4]:
NetDict = {'TimeAggNet':TimeAggNet,'TimeGraphNet':TimeGraphNet,'DeepTimeGraphNet':DeepTimeGraphNet,
           'SimpleTimeGraphNet':SimpleTimeGraphNet}

In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
subj = 0
j = 0
epochs = 250

for net_name in tqdm(NetDict.keys()):
    net = NetDict[net_name](device, DataClass.num_nodes, 1200)#El argumento cambia para las adaptative
    model_path = f"./training/models/sub{subj}/{net_name}.pth"
    metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}.csv"
    train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                path_metrics=metrics_path, verbose=False)
    
    print(f"Finished training model {net_name}.")
    if j>=1:
        net = NetDict[net_name](device, DataClass.num_nodes, 1200, True)#El argumento cambia para las adaptative
        model_path = f"./training/models/sub{subj}/{net_name}Adap.pth"
        metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}Adap.csv"
        train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                    path_metrics=metrics_path, verbose=False)
        print(f"Finished training model {net_name}Adap.")
        
    j = j+1

 25%|████████████████████▌                                                             | 1/4 [19:32<58:38, 1172.71s/it]

Finished training model TimeAggNet.


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages

Finished training model TimeGraphNet.


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 50%|███████████████████████████████████████                                       | 2/4 [1:30:23<1:39:27, 2983.57s/it]

...

Finished training model DeepTimeGraphNetAdap.


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages

Finished training model SimpleTimeGraphNet.


100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [5:26:16<00:00, 4894.11s/it]

Finished training model SimpleTimeGraphNetAdap.


* TimeGraphNet y SimpleTimeGraphNet no han funcionado adecuadamente y sus versiones adapt tampoco?.
* DeepTimeGraphNetAdap puede que tenga demasiados parametros tarda muchisimo.

### Cheking out problematic models

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
subj = 0
epochs = 250
problematic = ["SimpleTimeGraphNet"]
for net_name in tqdm(problematic):    
    net = NetDict[net_name](device, DataClass.num_nodes, 1200, True)#El argumento cambia para las adaptative
    model_path = f"./training/models/sub{subj}/{net_name}Adap.pth"
    metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}Adap.csv"
    train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                path_metrics=metrics_path, verbose=False)
    print(f"Finished training model {net_name}Adap.")

## Training by subject

In [4]:
NetDict = {'TimeAggNet':TimeAggNet,'TimeGraphNet':TimeGraphNet,'DeepTimeGraphNet':DeepTimeGraphNet,
           'SimpleTimeGraphNet':SimpleTimeGraphNet}

In [10]:
NetDict = {'TimeGraphNet':TimeGraphNet,'DeepTimeGraphNet':DeepTimeGraphNet}

In [11]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
subj = 10
j = 0
epochs = 250

DataClass = subjecData()
DataClass.build(subj)
for net_name in tqdm(NetDict.keys()):
    net = NetDict[net_name](device, DataClass.num_nodes, 1200)#El argumento cambia para las adaptative
    model_path = f"./training/models/sub{subj}/{net_name}.pth"
    metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}.csv"
    train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                path_metrics=metrics_path, verbose=False)
    
    print(f"Finished training model {net_name}.")
    if j>=1:
        net = NetDict[net_name](device, DataClass.num_nodes, 1200, True)#El argumento cambia para las adaptative
        model_path = f"./training/models/sub{subj}/{net_name}Adap.pth"
        metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}Adap.csv"
        train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                    path_metrics=metrics_path, verbose=False)
        print(f"Finished training model {net_name}Adap.")
        
    j = j+1

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this b

Finished training model TimeGraphNet.


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished training model DeepTimeGraphNet.


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages

Finished training model DeepTimeGraphNetAdap.


## Retraining Faulty models

In [15]:
NetDict = {'DeepTimeGraphNet':DeepTimeGraphNet}

In [6]:
faulty = {#1:{'DeepTimeGraphNet': (DeepTimeGraphNet, True)},
          #2:{'DeepTimeGraphNet': (DeepTimeGraphNet, True)},
          9:{'TimeGraphNet': (TimeGraphNet, False), 'DeepTimeGraphNet': (DeepTimeGraphNet, False)}
         }

In [17]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
epochs = 250

for subj in tqdm([9,10]): 
    print(f"Subject {subj}")
    DataClass = subjecData()
    DataClass.build(subj)
    j = 0
    for net_name in tqdm(NetDict.keys()):
        net = NetDict[net_name](device, DataClass.num_nodes, 1200, True)#El argumento cambia para las adaptative
        model_path = f"./training/models/sub{subj}/{net_name}Adap.pth"
        metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}Adap.csv"
        train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                    path_metrics=metrics_path, verbose=True, learning_rate = 0.01) #Maybe 0.0005
        print(f"Finished training model {net_name}Adap.")
        if j>=1:
            net = NetDict[net_name](device, DataClass.num_nodes, 1200, True)#El argumento cambia para las adaptative
            model_path = f"./training/models/sub{subj}/{net_name}Adap.pth"
            metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}Adap.csv"
            train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                        path_metrics=metrics_path, verbose=False, learning_rate = 0.01)
            print(f"Finished training model {net_name}Adap.")
        
    j = j+1

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Subject 9



  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

  0%|                                                                                          | 0/250 [00:00<?, ?it/s]

  0%|▎                                                                               | 1/250 [00:18<1:16:10, 18.36s/it]

Epoch 1, Training Loss: 0.22481123335433728, Accuracy: 0.32722007722007723, Precision: 0.32933111808914267, Recall: 0.327640695100476, F1-Score: 0.315505267353443




  1%|▋                                                                               | 2/250 [00:36<1:15:23, 18.24s/it]

...

Epoch 248, Training Loss: 0.2233245779167522, Accuracy: 0.31467181467181465, Precision: 0.20490942028985507, Recall: 0.31516263431852204, F1-Score: 0.23717537733508884




100%|█████████████████████████████████████████████████████████████████████████████▋| 249/250 [1:26:32<00:20, 20.62s/it]

Epoch 249, Training Loss: 0.22262084890495648, Accuracy: 0.32432432432432434, Precision: 0.3142576847612819, Recall: 0.3247658197909959, F1-Score: 0.2980003867451168




100%|██████████████████████████████████████████████████████████████████████████████| 250/250 [1:26:53<00:00, 20.85s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [2:52:19<00:00, 5169.84s/it]

Epoch 250, Training Loss: 0.22243486976984775, Accuracy: 0.30212355212355213, Precision: 0.2967287201281104, Recall: 0.30197402778627663, F1-Score: 0.2896774976727216
Finished training model DeepTimeGraphNetAdap.


In [18]:
NetDict = {'DeepTimeGraphNet':DeepTimeGraphNet}

In [19]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
subj = 6
epochs = 250
NetDict = {'DeepTimeGraphNet':DeepTimeGraphNet}
DataClass = subjecData()
DataClass.build(subj)
#learning_rate = 0.001
for net_name in tqdm(NetDict.keys()):
    net = NetDict[net_name](device, DataClass.num_nodes, 1200)#El argumento cambia para las adaptative
    model_path = f"./training/models/sub{subj}/{net_name}.pth"
    metrics_path = f"./training/metrics/sub{subj}/metrics{net_name}.csv"
    train_model(net, DataClass, num_epochs = epochs, save=True, path=model_path, 
                path_metrics=metrics_path, verbose=True, learning_rate = 0.01)
    
    print(f"Finished training model {net_name}.")

  0%|                                                                                          | 0/250 [00:00<?, ?it/s]C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  0%|▎                                                                               | 1/250 [00:15<1:02:27, 15.05s/it]

Epoch 1, Training Loss: 0.21246568858623505, Accuracy: 0.4027777777777778, Precision: 0.26976799242424243, Recall: 0.3299393507726841, F1-Score: 0.29431844124826584


C:\Users\Alejandro\anaconda3\envs\TFM\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

  1%|▋                                                                               | 2/250 [00:30<1:04:21, 15.57s/it]

Epoch 2, Training Loss: 0.21175917762297172, Accuracy: 0.3819444444444444, Precision: 0.2542307575130332, Recall: 0.3129829275662609, F1-Score: 0.2803204233466825



  1%|▉                                                                               | 3/250 [00:47<1:05:45, 15.97s/it]

...

Epoch 248, Training Loss: 0.13815018672634055, Accuracy: 0.6574074074074074, Precision: 0.6271376876535754, Recall: 0.5767065060543322, F1-Score: 0.5753642589846563



100%|█████████████████████████████████████████████████████████████████████████████▋| 249/250 [1:12:28<00:17, 17.30s/it]

Epoch 249, Training Loss: 0.13933547448228906, Accuracy: 0.6539351851851852, Precision: 0.625746232139362, Recall: 0.5637547838634794, F1-Score: 0.5552554290736811



100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [1:12:46<00:00, 4366.29s/it]

Epoch 250, Training Loss: 0.1409848321367193, Accuracy: 0.6458333333333334, Precision: 0.6356984398418054, Recall: 0.5548683569516902, F1-Score: 0.5456207877085377
Finished training model DeepTimeGraphNet.
